# Getting started with Azure Cosmos DB's API for MongoDB and Synapse Link

In this sample we will execute the following tasks:

1. Insert a dataset using the traditional MongoDB client.
1. Execute aggregation queries against the Analytical Store from the transactional data we inserted.
1. Insert another dataset, but this time using the MongoSpark connector.
1. Execute aggregation queries again, consolidating both datasets.

## Pre-requisites
1. Have you created a MongoDB API account in Azure Cosmos DB? If not, go to [Create an account for Azure Cosmos DB's API for MongoDB]().
1. For your Cosmos DB account, have you enabled Synapse Link? If not, go to [Enable Synapse Link for Azure Cosmos DB's API for MongoDB]().
1. Have you created a Synapse Workspace? If not, go to [Create Synapse Workspace account](). Please don't forget to add yourself as **Storage Blob Data Contributor** to the primary ADLS G2 account that is linked to the Synapse workspace.

## Create a Cosmos DB collection with Synapse Link

Please be careful, all commands are case sensitive.

1. Create a database named `DemoSynapseLinkMongoDB`. 
1. Create a collection named `HTAP` with a partition key called `item`. Make sure you set the `Analytical store` option to `On` when you create your collection.

## Optional - Connect your collection to Synapse

To accelerate future work, you can connect your collection to Synapse. **We won't use this capability in this demo**, but fell free to test and use it.

1. Go to your Synapse Analytics workspace.
1. Create a `Linked Data` connection for your MongoDB API account. 
    1. Under the `Data` blade, select the + (plus) sign.
    1. Select the `Connect to external data` option.
    1. Now select the `Azure Cosmos DB (MongoDB API)` option. 
    1. Enter all the information regarding your specific Azure Cosmos DB account either by using the dropdowns or by entering the connection string. Take note of the name you assigned to your `Linked Data` connection. 
    - Alternatively, you can also use the connection parameters from your account overview.
1. Test the connection by looking for your database accounts in the `Data` blade, and under the `Linked` tab.
    - There should be a list that contains all accounts and collections.
    - Collections that have an `Analytical Store` enabled will have a distinctive icon.

### Let's get the environment ready

This environment allows you to install and use any python libraries that you want to run. For this sample, you need to add the following libraries to your Spark pool:

```
pymongo==2.8.1
aenum==2.2.4
bson==0.5.10
```

Learn how to import libraries into your Spark pools in [this article](https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-azure-portal-add-libraries). Please use the `requirements.txt` file located in the same folder of this notebook to update your pool packages.

You can execute the following command to make sure all the libraries are installed correctly:

In [65]:
import importlib

packages = ['pymongo','bson','aenum']
for package in packages:
    test = importlib.util.find_spec(package)
    if test:
        print(package, "OK")
    else:
        print(package, "NOK")

StatementMeta(rosouzMongo, 25, 67, Finished, Available)

pymongo OK
bson OK
aenum OK
backports-abc NOK

### Add your database account and collection details here!

In [66]:
DATABASE_ACCOUNT_NAME = 'rosouz-synapse-link-mongodb-demo'
DATABASE_ACCOUNT_READWRITE_KEY = 'uov8BOLgmwXtY8lGh96RVKIPsFuoKBcbpU6y9Zw1LSimMWJ68Itqw98SGYeIh8It2HRM6jd5KONSiaKuB8MMPQ=='
DATABASE_NAME = 'DemoSynapseLinkMongoDB'
COLLECTION_NAME = 'HTAP'

StatementMeta(rosouzMongo, 25, 68, Finished, Available)

## Let's initialize the MongoDB client

You are only going to need the following parameters from your account overview: 
- Connection string.
- Primary or secondary ready/write key.

Remember that we named our database `DemoSynapseLinkMongoDB` and our collection `HTAP`.

The code snippet below shows how to initialize the `MongoClient` object.

In [67]:
from pymongo import MongoClient
from bson import ObjectId # For ObjectId to work

client = MongoClient("mongodb://{account}.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb".format(account = DATABASE_ACCOUNT_NAME)) # Your own database account endpoint.
db = client.DemoSynapseLinkMongoDB    # Select the database
db.authenticate(name=DATABASE_ACCOUNT_NAME,password=DATABASE_ACCOUNT_READWRITE_KEY) # Use your database account name and any of your read/write keys.

StatementMeta(rosouzMongo, 25, 69, Finished, Available)

True

## Inserting data with the MongoClient driver

The following sample will generate 500 items based on random data. Each item will contain the following fields:
- item, string
- price, float
- rating, integer
- timestamp, [epoch integer](http://unixtimestamp.50x.eu/about.php)

This data will be inserted into the MongoDB store of your database. This emulates the transactional data that an application would generate.

In [68]:
from random import randint
import time

orders = db["HTAP"]

items = ['Pizza','Sandwich','Soup', 'Salad', 'Tacos']
prices = [2.99, 3.49, 5.49, 12.99, 54.49]

for x in range(1, 501):
    order = {
        'item' : items[randint(0, (len(items)-1))],
        'price' : prices[randint(0, (len(prices)-1))],
        'rating' : randint(1, 5),
        'timestamp' : time.time()
    }
    
    result=orders.insert(order)

print('finished creating 500 orders')


StatementMeta(rosouzMongo, 25, 70, Finished, Available)

finished creating 500 orders

## Read data from the Analytical Store.

Now that we have inserted some transactional data, let's read it from Azure Cosmos DB analytical store. Cosmos DB will automatically transform the BSON data (Binary JSON) into a columnar format, which will make it fast and easy to execute aggregation workloads on top of your transactional data, at no RUs or performance costs.

The cells below will:

1. Load the data from analytical store into a DataFrame.
1. Check the top 10 rows. Yes, the BSON data was converted into columar structured format.
1. Run aggregations
1. Check the DataFrame schema.


> If you get an "no snapshot" error, please wait a couple of minutes because the root cause is that the auto sync between transactional and analytical stores isn't completed yet. This process usually takes up to 2 minutes, but in some cases it may take up to 5 minutes.

**Important: Please note that we are using random values for prices and ratings. Don't expect the same results of the outputs below. What you can expect is the same behavior and experience.**


In [69]:
# Load the data from analytical store into a DataFrame.
df = spark.read.format("cosmos.olap")\
    .option("spark.cosmos.accountEndpoint", "https://{account}.documents.azure.com:443/".format(account = DATABASE_ACCOUNT_NAME))\
    .option("spark.cosmos.accountKey", DATABASE_ACCOUNT_READWRITE_KEY)\
    .option("spark.cosmos.database", DATABASE_NAME)\
    .option("spark.cosmos.container", COLLECTION_NAME)\
    .load()

# Check the top 10 rows
df.head(10)

StatementMeta(rosouzMongo, 25, 71, Finished, Available)

[Row(_rid='JYAqAKy0s3+eDAAAAAAAAA==', _ts=1603308932, id='NWY5MDhkODQ2NWYwNjMzYjI4Y2VhMGUz', _etag='"17003d58-0000-0800-0000-5f908d840000"', _id=Row(objectId='_???e?c;(??'), item=Row(string='Salad'), price=Row(float64=54.49), rating=Row(int32=5), timestamp=Row(float64=1603308932.6953204, string=None), _partitionKey=Row(string='JYAqAKy0s38=')), Row(_rid='JYAqAKy0s3+fDAAAAAAAAA==', _ts=1603308932, id='NWY5MDhkODQ2NWYwNjMzYjI4Y2VhMGU0', _etag='"17003e58-0000-0800-0000-5f908d840000"', _id=Row(objectId='_???e?c;(??'), item=Row(string='Salad'), price=Row(float64=12.99), rating=Row(int32=4), timestamp=Row(float64=1603308932.766245, string=None), _partitionKey=Row(string='JYAqAKy0s38=')), Row(_rid='JYAqAKy0s3+gDAAAAAAAAA==', _ts=1603308932, id='NWY5MDhkODQ2NWYwNjMzYjI4Y2VhMGU1', _etag='"17003f58-0000-0800-0000-5f908d840000"', _id=Row(objectId='_???e?c;(??'), item=Row(string='Pizza'), price=Row(float64=12.99), rating=Row(int32=2), timestamp=Row(float64=1603308932.7732844, string=None), _partiti

In [70]:
# Run aggregations

df.groupBy(df.item.string).sum().show()

StatementMeta(rosouzMongo, 25, 72, Finished, Available)

+------------+-------------+
|item[string]|     sum(_ts)|
+------------+-------------+
|       Salad|1566430357101|
|       Tacos|1539174145337|
|    Sandwich|1704314980807|
|       Pizza|1593686821692|
|        Soup|1612926241703|
+------------+-------------+

In [71]:
# Check the DataFrame schema

df.printSchema

StatementMeta(rosouzMongo, 25, 73, Finished, Available)

<bound method DataFrame.printSchema of DataFrame[_rid: string, _ts: bigint, id: string, _etag: string, _id: struct<objectId:string>, item: struct<string:string>, price: struct<float64:double>, rating: struct<int32:int>, timestamp: struct<float64:double,string:string>, _partitionKey: struct<string:string>]>

## Schema Representation - A quick note about the MongoDB schema in analytical store

Please note in the result above that for the `timestamp` field we have only 1 datatype: `struct<float64:double>`. 
We will see that this detail will change since we will insert data with different datatype for that `timestamp` field.

For MongoDB accounts we make use of a **Full Fidelity Schema** as a default option. This is a representation of property names extended with their data types to provide an accurate 
representation of their values and avoid ambiguity.

This is why, when we called the fields above, we used their datatype as a suffix. Like in the example below:

```
df.filter((df.item.string == "Pizza")).show(10)
```

Notice how we specified the `string` type after the name of the property. Here is a map of all potential properties and their suffix representations in the Analytical Store:

| Original Data Type     &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Suffix    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Example &nbsp;&nbsp;&nbsp;&nbsp; | 
|---------------|----------------|--------|
| Double        | ".float64"     |  `24.99`   |
| Array         | ".array"       |  `["a", "b"]`   |
| Binary        | ".binary"      |  `0`   |
| Boolean       | ".bool"        |  `True`   |
| Int32         | ".int32"       |  `123`   |
| Int64         | ".int64"       |  `255486129307`   |
| Null          | ".null"        |  `null`   |
| String        | ".string"      |  `"ABC"`   |
| Timestamp     | ".timestamp"   |  `Timestamp(0, 0)`   |
| DateTime      | ".date"        |  `ISODate("2020-08-21T07:43:07.375Z")`   |
| ObjectId      | ".objectId"    |  `ObjectId("5f3f7b59330ec25c132623a2")`   |
| Document      | ".object"      |  `{"a": "a"}`   |

These types are inferred from the data that is inserted in the transactional store. You can see the schema by executing the following command:
```
df.printSchema
```



For more information about schemas representation, click [here](https://docs.microsoft.com/azure/cosmos-db/analytical-store-introduction#schema-representation) .



## Let's insert more orders!

This time we will use slightly different data. Each item will contain the following fields:
- item, string
- price, float
- rating, integer
- timestamp, [ISO String format](https://en.wikipedia.org/wiki/ISO_8601)

Notice how the `Timestamp` field is now in a string format. This will help us understand how the different data fields can be read based on their data type.

After that, we will load the data, check the schema, and run some queries.

In [72]:
from random import randint
from time import strftime

orders = db["HTAP"]

items = ['Pizza','Sandwich','Soup', 'Salad', 'Tacos']
prices = [2.99, 3.49, 5.49, 12.99, 54.49]

for x in range(1, 501):
    order = {
        'item' : items[randint(0, (len(items)-1))],
        'price' : prices[randint(0, (len(prices)-1))],
        'rating' : randint(1, 5),
        'timestamp' : strftime("%Y-%m-%d %H:%M:%S")
    }
    
    result=orders.insert(order)

print('finished creating 500 orders')

StatementMeta(rosouzMongo, 25, 74, Finished, Available)

finished creating 500 orders

## Let's read the data and check the schema again!

In [73]:
# Load the Analytical Store data into a dataframe
# Make sure to run the cell with the secrets to get the DATABASE_ACCOUNT_NAME and the DATABASE_ACCOUNT_READWRITE_KEY variables.
df = spark.read.format("cosmos.olap")\
    .option("spark.cosmos.accountEndpoint", "https://{account}.documents.azure.com:443/".format(account = DATABASE_ACCOUNT_NAME))\
    .option("spark.cosmos.accountKey", DATABASE_ACCOUNT_READWRITE_KEY)\
    .option("spark.cosmos.database", DATABASE_NAME)\
    .option("spark.cosmos.container", COLLECTION_NAME)\
    .load()

# Check the schema AGAIN. Try to find something different.
df.printSchema

StatementMeta(rosouzMongo, 25, 75, Finished, Available)

<bound method DataFrame.printSchema of DataFrame[_rid: string, _ts: bigint, id: string, _etag: string, _id: struct<objectId:string>, item: struct<string:string>, price: struct<float64:double>, rating: struct<int32:int>, timestamp: struct<float64:double,string:string>, _partitionKey: struct<string:string>]>

## Schema Representation - What Changed?

Please note in the result above that now, for the `timestamp` field, we have 2 datatypes: `struct<float64:double>` and `string:string`. That happened because we added data with a different datatype. That's `Full Fidelity Schema`, Azure Cosmos DB will do a full representation of your data, with the datatypes you used.

## Queries

Now let's run some interesting queries, using the datypes to filter the data.


In [74]:
# SQL!!
# Let's see the data for pizzas that have a string timestamp
df.createOrReplaceTempView("Pizza")
sql_results = spark.sql("SELECT sum(price.float64),count(*) FROM Pizza where timestamp.string is not null and item.string = 'Pizza'")
sql_results.show()

StatementMeta(rosouzMongo, 25, 76, Finished, Available)

+-------------------------------+--------+
|sum(price.float64 AS `float64`)|count(1)|
+-------------------------------+--------+
|              6125.269999999993|     423|
+-------------------------------+--------+

In [75]:
# SQL!!
# Let's see the data for pizzas that have a string timestamp
df.createOrReplaceTempView("Pizza")
sql_results = spark.sql("SELECT sum(price.float64),count(*) FROM Pizza where timestamp.float64 is not null and item.string = 'Pizza'")
sql_results.show()

StatementMeta(rosouzMongo, 25, 77, Finished, Available)

+-------------------------------+--------+
|sum(price.float64 AS `float64`)|count(1)|
+-------------------------------+--------+
|              9014.289999999995|     571|
+-------------------------------+--------+

In [76]:
# SQL!!
# Let's compare both timestamp columns
df.createOrReplaceTempView("Pizza")
sql_results = spark.sql("SELECT max(timestamp.float64),max(timestamp.string) FROM Pizza where item.string = 'Pizza'")
sql_results.show()

StatementMeta(rosouzMongo, 25, 78, Finished, Available)

+-----------------------------------+---------------------------------+
|max(timestamp.float64 AS `float64`)|max(timestamp.string AS `string`)|
+-----------------------------------+---------------------------------+
|                1.603308993772331E9|              2020-10-21 19:36:56|
+-----------------------------------+---------------------------------+

## Schema Representation - Last thoughts

Please note that the queries above return different data because of the filters on the timestamp column. From the user perspective, it's like there are 2 different columns, `timestamp.float64` and `timestamp.string`.

## Conclusion

Now you know how to use Azure Synapse Link for Azure Cosmos DB analitical store for MongoDB API. Also, now you know how to work with dataframes, full fidelity schema, and Spark Sql.